# 🎵 Spotify End-to-End Data Pipeline using Databricks (PySpark) + Snowflake + AWS S3

This notebook is part of an end-to-end ETL project that ingests raw Spotify track data from the Spotify API into AWS S3, applies transformations using Databricks (PySpark), and loads the final curated dataset into Snowflake for analytics and BI reporting.

*Tech Stack:* Python, PySpark, Databricks, AWS S3, Snowflake, SQL  
*Pipeline Objective:* Build a scalable, production-style data pipeline for music analytics.

## 📌 Step 1 — Load processed CSV from AWS S3 (via secure pre-signed URL)

In [0]:
import pandas as pd

presigned_url = "https://mani-spotify-etl-data.s3.us-east-2.amazonaws.com/spotify/processed/tracks_transformed_20251119_214719.csv?
df_pd = pd.read_csv(presigned_url)

df_pd.head()

,artist,album_name,track_name,track_id,duration_ms,explicit,duration_minutes,length_category,album_track_count,album_popularity_rank
0,The Weeknd,Hurry Up Tomorrow,Wake Me Up (feat. Justice),5673WA8EEUSPx1ir26lhGW,308575,False,5.14,Long (>5 min),22,4
1,The Weeknd,Hurry Up Tomorrow,Cry For Me,3AWDeHLc88XogCaCnZQLVI,224136,True,3.74,Medium (3-5 min),22,4
2,The Weeknd,Hurry Up Tomorrow,I Can't Fucking Sing,64JIAZ0bS7WoARYfWQGCoz,12288,True,0.20,Short (<3 min),22,4
3,The Weeknd,Hurry Up Tomorrow,São Paulo (feat. Anitta),7DY756WOLyOz2Xnhw4EFiC,301623,True,5.03,Long (>5 min),22,4
4,The Weeknd,Hurry Up Tomorrow,Until We're Skin & Bones,6jDGDtQPC46pFqxph3qdbD,22033,False,0.37,Short (<3 min),22,4


## 2️⃣ Convert Pandas DataFrame to Spark DataFrame
We convert the dataset into a distributed Spark DataFrame to enable scalable processing and parallel computation.

In [0]:
df = spark.createDataFrame(df_pd)
df.show(10)
df.printSchema()

+----------+-----------------+--------------------+--------------------+-----------+--------+----------------+----------------+-----------------+---------------------+
|    artist|       album_name|          track_name|            track_id|duration_ms|explicit|duration_minutes| length_category|album_track_count|album_popularity_rank|
+----------+-----------------+--------------------+--------------------+-----------+--------+----------------+----------------+-----------------+---------------------+
|The Weeknd|Hurry Up Tomorrow|Wake Me Up (feat....|5673WA8EEUSPx1ir2...|     308575|   false|            5.14|   Long (>5 min)|               22|                    4|
|The Weeknd|Hurry Up Tomorrow|          Cry For Me|3AWDeHLc88XogCaCn...|     224136|    true|            3.74|Medium (3-5 min)|               22|                    4|
|The Weeknd|Hurry Up Tomorrow|I Can't Fucking Sing|64JIAZ0bS7WoARYfW...|      12288|    true|             0.2|  Short (<3 min)|               22|               

## 3️⃣ Add New Columns — Feature Engineering
We derive new attributes for better analytics:
- duration_minutes = duration_ms converted to minutes
- length_category = Long / Medium / Short based on duration

In [0]:
from pyspark.sql.functions import *

df = df.withColumn("duration_minutes", col("duration_ms") / 60000)
df = df.withColumn(
    "length_category",
    when(col("duration_minutes") > 5, "Long (>5 min)")
    .when(col("duration_minutes") >= 3, "Medium (3-5 min)")
    .otherwise("Short (<3 min)")
)

df.show(10)
df.printSchema()

+----------+-----------------+--------------------+--------------------+-----------+--------+------------------+----------------+-----------------+---------------------+
|    artist|       album_name|          track_name|            track_id|duration_ms|explicit|  duration_minutes| length_category|album_track_count|album_popularity_rank|
+----------+-----------------+--------------------+--------------------+-----------+--------+------------------+----------------+-----------------+---------------------+
|The Weeknd|Hurry Up Tomorrow|Wake Me Up (feat....|5673WA8EEUSPx1ir2...|     308575|   false| 5.142916666666666|   Long (>5 min)|               22|                    4|
|The Weeknd|Hurry Up Tomorrow|          Cry For Me|3AWDeHLc88XogCaCn...|     224136|    true|            3.7356|Medium (3-5 min)|               22|                    4|
|The Weeknd|Hurry Up Tomorrow|I Can't Fucking Sing|64JIAZ0bS7WoARYfW...|      12288|    true|            0.2048|  Short (<3 min)|               22|   

📊 Step 4 — Exploratory Spark Analysis

Track count and popularity by length category

In [0]:
(
    df.groupBy("length_category")
      .agg(
          count("*").alias("track_count"),
          round(avg("duration_minutes"), 2).alias("avg_duration_min"),
          round(avg("album_popularity_rank"), 2).alias("avg_album_pop_rank")
      )
      .orderBy("length_category")
      .show()
)

+----------------+-----------+----------------+------------------+
| length_category|track_count|avg_duration_min|avg_album_pop_rank|
+----------------+-----------+----------------+------------------+
|   Long (>5 min)|         58|            5.99|              6.52|
|Medium (3-5 min)|        170|            3.86|              5.63|
|  Short (<3 min)|         34|            2.16|              4.09|
+----------------+-----------+----------------+------------------+



In [0]:
(
    df.select("artist", "album_name", "track_name", "duration_minutes", "length_category")
      .orderBy(col("duration_minutes").desc())
      .show(10, truncate=False)
)

+----------+----------------------------+---------------------------+------------------+---------------+
|artist    |album_name                  |track_name                 |duration_minutes  |length_category|
+----------+----------------------------+---------------------------+------------------+---------------+
|The Weeknd|Thursday (Original)         |Gone                       |8.107083333333334 |Long (>5 min)  |
|The Weeknd|Trilogy                     |Gone                       |8.107083333333334 |Long (>5 min)  |
|The Weeknd|Trilogy                     |The Party & The After Party|7.660983333333333 |Long (>5 min)  |
|The Weeknd|House Of Balloons (Original)|The Party & The After Party|7.660983333333333 |Long (>5 min)  |
|The Weeknd|Kiss Land                   |Kiss Land                  |7.588883333333333 |Long (>5 min)  |
|The Weeknd|Kiss Land                   |Tears In The Rain          |7.4108833333333335|Long (>5 min)  |
|The Weeknd|Trilogy                     |XO / The Host 

❄ Step 5 — (Planned) Write to Snowflake — Production Version

Snowflake writeback uses the Snowflake Spark Connector, which is not supported on Databricks Free Serverless.
However, below is the production-ready code (to be run on a non-serverless cluster):

In [0]:
# sf_options = {
#     "sfURL": "<account>.snowflakecomputing.com",
#     "sfUser": "<user>",
#     "sfPassword": "<password>",
#     "sfWarehouse": "SPOTIFY_WH",
#     "sfDatabase": "SPOTIFY_ETL_DB",
#     "sfSchema": "SPOTIFY_SCHEMA"
# }

# df.write \
#   .format("snowflake") \
#   .options(**sf_options) \
#   .option("dbtable", "SPOTIFY_TRACKS_PROCESSED") \
#   .mode("overwrite") \
#   .save()

📌 *Note — Snowflake Load Already Completed Outside Databricks*

In this project, the final load into Snowflake was executed earlier using the Snowflake COPY INTO command to ingest data directly from the AWS S3 bucket.

Therefore, the df.write.format("snowflake") block is commented out in this notebook to avoid duplicate loads.

```sql
COPY INTO SPOTIFY_ETL_DB.SPOTIFY_SCHEMA.SPOTIFY_TRACKS_PROCESSED
FROM 's3://mani-spotify-etl-data/spotify/processed/'
FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY='"' SKIP_HEADER = 1)
PATTERN = 'tracks_transformed_.*.csv'
;

### ✔ Pipeline Completed Successfully

This notebook completes the Databricks transformation stage of the pipeline.
We securely loaded transformed Spotify track data from AWS S3, engineered new features
(duration_minutes, length_category), and analyzed track characteristics at scale using PySpark.

Next stage — Data is made available in Snowflake for BI dashboards and analytics (Power BI / Tableau).